# **Classification** des **séries temporelles** avec `sktime`

## Métadonnées

- **Expérience nº :** 8.2.2
- **Date :** 11/11/2023
- **Heure :** 12:32
- **Données :** `ready_Sunshine.csv`
- **Tri :** index chronologique
- **Découpage :** `TimeSeriesSplit`
- **Mise à l'échelle :** non
- **Rééquilibrage :** non
- **Conversion :** `numpyfy`
- **Algorithme :** TSF

## Sommaire

1. Initialisation
2. Découpage
3. Conversion
4. Modélisation
5. Évaluation

## 1. Initialisation

In [1]:
# Importation des bibliothèques et modules nécessaires au fonctionnement de ce notebook

import pandas as pd
import numpy as np

from sklearn.metrics import classification_report

In [2]:
# Importation du jeu de données et enregistrement dans le DataFrame `df`

df = pd.read_csv("../../../data/9df/Fred/ready_Sunshine.csv", index_col = 1).sort_index()
df = df.drop(columns = "Unnamed: 0")

In [3]:
# Inspection de la structure de `df`

df.head()

,MinTemp,MaxTemp,Rainfall,Evaporation,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,...,year,month,day,LocationNum,WindGustDirNum,WindDir9amNum,WindDir3pmNum,Latitude,Longitude,CodeRegionNum
Date,,,,,,,,,,,,,,,,,,,,,
2008-12-01,9.1,21.7,0.0,7.4,44.0,20.0,20.0,47.0,23.0,1009.1,...,2008,12,1,19,3.141593,2.748894,3.534292,-36.758877,144.282593,2
2008-12-01,9.5,17.9,3.4,15.2,57.0,19.0,28.0,53.0,61.0,1000.6,...,2008,12,1,44,2.748894,2.748894,2.356194,-41.434081,147.137350,6
2008-12-02,9.4,18.3,4.0,6.0,46.0,24.0,28.0,60.0,46.0,1001.4,...,2008,12,2,44,2.748894,2.356194,2.356194,-41.434081,147.137350,6
2008-12-02,6.2,23.0,0.0,8.8,48.0,20.0,26.0,49.0,29.0,1010.4,...,2008,12,2,19,3.141593,2.748894,2.748894,-36.758877,144.282593,2
2008-12-03,9.1,18.4,1.0,5.8,50.0,17.0,24.0,62.0,37.0,1002.4,...,2008,12,3,44,3.534292,2.356194,2.748894,-41.434081,147.137350,6


In [4]:
# Inspection de la structure de `df`

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10501 entries, 2008-12-01 to 2017-06-25
Data columns (total 27 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   MinTemp         10501 non-null  float64
 1   MaxTemp         10501 non-null  float64
 2   Rainfall        10501 non-null  float64
 3   Evaporation     10501 non-null  float64
 4   WindGustSpeed   10501 non-null  float64
 5   WindSpeed9am    10501 non-null  float64
 6   WindSpeed3pm    10501 non-null  float64
 7   Humidity9am     10501 non-null  float64
 8   Humidity3pm     10501 non-null  float64
 9   Pressure9am     10501 non-null  float64
 10  Pressure3pm     10501 non-null  float64
 11  Cloud9am        10501 non-null  float64
 12  Cloud3pm        10501 non-null  float64
 13  Temp9am         10501 non-null  float64
 14  Temp3pm         10501 non-null  float64
 15  RainToday       10501 non-null  int64  
 16  RainTomorrow    10501 non-null  int64  
 17  year            10501 

## 2. Découpage

In [5]:
# Découpage de `df` sur l'axe des colonnes : séparation des variables explicatives (`data`) et cible (`target`)

data = df.drop(columns = "RainTomorrow")
target = df["RainTomorrow"]

In [6]:
# Découpage de `data` et de `target` sur l'axe des lignes : séparation des jeux d'entraînement (`*_train`) et de test (`*_test`) avec le splitter `TimeSeriesSplit`

from sklearn.model_selection import TimeSeriesSplit

tss = TimeSeriesSplit(n_splits = 5) ## Nous fixons le paramètre `n_splits` à 5 afin d'avoir une répartition de 80 / 20 entre les jeux d'entraînement et de test, respectivement.

for train_index, test_index in tss.split(data):
    X_train, X_test = data.iloc[train_index, :], data.iloc[test_index,:]
    y_train, y_test = target.iloc[train_index], target.iloc[test_index]

## 3. Conversion

In [7]:
# Importation de la fonction artisanale `numpyfy`

from numpyfy import numpyfy

In [8]:
# Application de la fonction `numpyfy` aux 4 jeux de données issus du découpage effectué par le splitter `TimeSeriesSplit` afin de les convertir en arrays et ainsi les rendre compatibles avec `sktime`

X_train, X_test, y_train, y_test = numpyfy(X_train, X_test, y_train, y_test)

## 4. Modélisation

In [9]:
# Instanciation d'un modèle classificateur via un Pipeline

from sklearn.pipeline import Pipeline
from sktime.transformations.panel.compose import ColumnConcatenator
from sktime.classification.interval_based import TimeSeriesForestClassifier

steps = [
    ("concatenate", ColumnConcatenator()), ## Cette étape permet de transformer les jeux de données multivariées en univariées afin qu'ils puissent être traités par `TimeSeriesForestClassifier`.
    ("classify", TimeSeriesForestClassifier()),
]

clf_rf_ts = Pipeline(steps)

In [10]:
# Récupération des paramètres initiaux de la TSF

clf_rf_ts["classify"].get_params()

{'min_interval': 3, 'n_estimators': 200, 'n_jobs': 1, 'random_state': None}

In [11]:
# Entraînement du modèle

clf_rf_ts.fit(X_train, y_train)

Pipeline(steps=[('concatenate', ColumnConcatenator()),
                ('classify', TimeSeriesForestClassifier())])

In [12]:
# Récupération des paramètres ajustés de la TSF

clf_rf_ts["classify"].get_fitted_params()

{'classes': array([0, 1]),
 'intervals': [array([[ 4, 13],
         [10, 19],
         [22, 25],
         [15, 19],
         [16, 19]]),
  array([[ 4,  7],
         [ 3, 23],
         [15, 22],
         [16, 19],
         [14, 24]]),
  array([[12, 17],
         [15, 22],
         [15, 18],
         [22, 25],
         [ 5,  8]]),
  array([[10, 14],
         [12, 19],
         [16, 23],
         [ 0,  4],
         [14, 17]]),
  array([[ 3,  7],
         [19, 22],
         [21, 24],
         [13, 21],
         [11, 19]]),
  array([[20, 23],
         [ 1,  8],
         [ 1, 24],
         [20, 23],
         [15, 18]]),
  array([[22, 25],
         [12, 17],
         [22, 25],
         [17, 22],
         [16, 24]]),
  array([[ 0,  4],
         [15, 24],
         [ 4, 20],
         [16, 19],
         [ 8, 12]]),
  array([[13, 17],
         [22, 25],
         [ 8, 15],
         [14, 21],
         [18, 22]]),
  array([[18, 21],
         [14, 19],
         [ 0, 23],
         [20, 24],
         [1

In [13]:
# Réalisation des prédictions

y_pred = clf_rf_ts.predict(X_test)

## 5. Évaluation

In [14]:
# Élaboration de la matrice de confusion

pd.crosstab(y_test, y_pred, rownames = ["Classe réelle"], colnames = ["Classe prédite"])

Classe prédite,0,1
Classe réelle,,
0,1361,22
1,258,109


In [15]:
# Élaboration du rapport de classification

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.84      0.98      0.91      1383
           1       0.83      0.30      0.44       367

    accuracy                           0.84      1750
   macro avg       0.84      0.64      0.67      1750
weighted avg       0.84      0.84      0.81      1750

